## Youtube 스크립트 텍스트 전처리

In [1]:
import pandas as pd
import numpy as np
from hanspell import spell_checker # 500글자 이상의 문장은 검사 불가
import re
import warnings 
warnings.filterwarnings('ignore')

from konlpy.tag import Okt
okt = Okt()

In [2]:
# db 연동
import pymysql

com_list = {'하이닉스' : '000660',
           '현대차' : '005380',
           '삼성전자' : '005930',
           'LG화학' : '051910',
           '셀트리온' : '068270'}
media_list = {'youtube_한국경제_':'hk_',
             'youtube_삼프로_':'sampro_',
             'youtube_슈카_':'suka_'}
df_list = []
for n,m in media_list.items():
    for i,j in com_list.items():

        db = pymysql.connect(
            user='root', 
            passwd='1234', 
            host='3.35.70.166', 
            db='proj', 
            charset = 'utf8'
        )

        cursor = db.cursor(pymysql.cursors.DictCursor)
        sql = "select * from youtube_"+ m + j +" where length(date) = 10"
        
        cursor.execute(sql)

        result = cursor.fetchall()

        # 데이터 프레임으로 변경
        globals() ['df_'+ m + j ] = pd.DataFrame(result)
        print('df_'+ m + j)
        df_list.append(globals() ['df_'+ m + j ])
        
        db.close()

df_hk_000660
df_hk_005380
df_hk_005930
df_hk_051910
df_hk_068270
df_sampro_000660
df_sampro_005380
df_sampro_005930
df_sampro_051910
df_sampro_068270
df_suka_000660
df_suka_005380
df_suka_005930
df_suka_051910
df_suka_068270


In [5]:
df1 = pd.concat(df_list)

In [2]:
path = input('절대 경로 입력 : ')

절대 경로 입력 :  C:\Users\j\stock_predict\Sentiment_Analysis\data\전처리 전 데이터\df_youtube_삼프로LG화학.csv


In [110]:
df1 = pd.read_csv(path)

In [6]:
df1.text

0      4 오랄 3 십일조를 판 외국인의지금 반도체를 향해서는 이달 들어서조금씩 매수를 해...
1      출발 중시 오늘도 진짜 전략으로시작하게 있는데요 오늘은 타임 경제이 순조 대표 모셨...
2      4 자 6 시장의 이슈 속에서중장기적으로 가져가며 4종목 때그리고 포트폴리오 전략을...
3      톡톡 놈 놈 놈 놈공시의 진단을 해보는 시간입니다 타키온 뉴스에 조호진 대표입니다안...
4      자 이번 프로 마케터 시간인데요 시장본을 시장을 보는 안목을 키우는 그런시간입니다 ...
                             ...                        
192    으 르3메인 얘기는 양적 만합니다 양쪽은 얘기를 왜 하느냐 끝날 때쯤에들어오시면 알...
193    아니 오기까지는 이해를 할 수 있잖아요 여기서 격이 도 그래도 좀인간적인 집에서 그...
194    [음악]가장 유명한 얘기부터 드릴께요 이해를 좀 쉽게 해야 되니까 2003년에있었던...
195    으 르3방송을 하다보면 은 듣는 이렇게 듣는 말 아니면 많이 뜨는 채 팀 아니면많이...
196    시골 쥐 님 안녕하세요 으1 으없이 많군요 예안녕하세요 소리 들리죠안녕하세요먹자프리...
Name: text, Length: 6385, dtype: object

In [7]:
# 데이터 전처리

df1.text = df1.text.str.replace('[으*]','')
df1.text = df1.text.str.replace('[\[\]]','')
df1.text = df1.text.str.replace('음악','')
df1.text = df1.text.str.replace('자','')
df1.text = df1.text.str.replace('[a-zA-Z]','')
df1.text = df1.text.str.replace('[구독*]','')
df1.text = df1.text.str.replace('[좋아요]','')
df1.text = df1.text.str.replace('[\s]','')
df1.text = df1.text.str.strip() # 앞뒤 공백 제거

df1.text 

0      4오랄3십일조를판외국인의지금반도체를향해서는이달들어서조금씩매수를해주고있습니다어떻게해석...
1      출발중시오늘도진짜전략로시작하게있는데오늘은타임경제이순조대표모셨습니다어서오세약간64일단...
2      46시장의이슈속에서중장기적로가져가며4종목때그리고포트폴리오전략을함께세워보도록하겠습니다...
3      톡톡놈놈놈놈공시의진단을해보는시간입니다타키온뉴스에조호진대표입니다안녕하세그러세4오늘은하...
4      이번프로마케터시간인데시장본을시장을보는안목을키우는그런시간입니다오늘은증권중소성장기업분석...
                             ...                        
192    르3메인얘기는양적만합니다양쪽은얘기를왜하느냐끝날때쯤에들어오시면알게될거에장점와나를알기전...
193    니오기까지는이해를할수있잖여기서격이도그래도좀인간적인집에서그래도나여기서부터약을먹지않고서...
194    가장유명한얘기부터드릴께이해를좀쉽게해야되니까2003년에있었던그룹의소버린공격사태입니다이...
195    르3방송을하다보면은듣는이렇게듣는말니면많이뜨는채팀니면많이뜬곱뭐라그러냐댓글같은겠어그1번...
196    시골쥐님안녕하세1없이많군예안녕하세소리들리죠안녕하세먹프리셀이시죠스피킹님안녕하세잠시만러...
Name: text, Length: 6385, dtype: object

In [8]:
# 사전 로딩

import codecs

positive = [] 
negative = []
posneg = []
dic = [('./KOSELF_pos.txt', './KOSELF_neg.txt'),
       ("./positive_words_self.txt","./negative_words_self.txt"),
      ('./sum_pos.txt','./sum_neg.txt')]

i = int(input('koself : 1, blog : 2, sum : 3'))-1


pos = codecs.open(dic[i][0], 'rb', encoding='UTF-8') 

while True: 
    line = pos.readline() 
    line = line.replace('\n', '')
    line = line.replace('\r', '')
    
    if not line: break
        
    positive.append(line) 
    posneg.append(line) 
        
pos.close() 

neg = codecs.open(dic[i][1], 'rb', encoding='UTF-8') 
while True: 
    line = neg.readline() 
    line = line.replace('\n', '')
    line = line.replace('\r', '')
    
    if not line: break
    
    negative.append(line) 
    posneg.append(line) 
    
neg.close()

koself : 1, blog : 2, sum : 3 3


In [9]:
# 불용어
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])

In [10]:
df1['token'] = ''

In [11]:
df2 = df1.copy()

In [12]:
import sys
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [125]:
# # test 10개만 빠르게 수행
# df2 = df1[:100]

In [14]:
# 토큰화
for i in range(len(df2)):
    temp = []
    checked = []
    
    temp = okt.nouns(df2['text'].iloc[i])
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    temp = [word for word in temp if len(word) > 1] # 불용어 제거
#     print(i,'번 토큰화, 불용어 제거 완료')
    
    temp = list(set(temp))
#     print(i, ' 중복단어 제거')
    for check in temp:
        aa = spell_checker.check(check).checked
        checked.append(aa)
#     print(i,'- spell check')
    
    df2['token'].iloc[i] = checked
    print(i,'- 완료')

0 - 완료
1 - 완료
2 - 완료
3 - 완료
4 - 완료
5 - 완료
6 - 완료
7 - 완료
8 - 완료
9 - 완료
10 - 완료
11 - 완료
12 - 완료
13 - 완료
14 - 완료
15 - 완료
16 - 완료
17 - 완료
18 - 완료
19 - 완료
20 - 완료
21 - 완료
22 - 완료
23 - 완료
24 - 완료
25 - 완료
26 - 완료
27 - 완료
28 - 완료
29 - 완료
30 - 완료
31 - 완료
32 - 완료
33 - 완료
34 - 완료
35 - 완료
36 - 완료
37 - 완료
38 - 완료
39 - 완료
40 - 완료
41 - 완료
42 - 완료
43 - 완료
44 - 완료
45 - 완료
46 - 완료
47 - 완료
48 - 완료
49 - 완료
50 - 완료
51 - 완료
52 - 완료
53 - 완료
54 - 완료
55 - 완료
56 - 완료
57 - 완료
58 - 완료
59 - 완료
60 - 완료
61 - 완료
62 - 완료
63 - 완료
64 - 완료
65 - 완료
66 - 완료
67 - 완료
68 - 완료
69 - 완료
70 - 완료
71 - 완료
72 - 완료
73 - 완료
74 - 완료
75 - 완료
76 - 완료
77 - 완료
78 - 완료
79 - 완료
80 - 완료
81 - 완료
82 - 완료
83 - 완료
84 - 완료
85 - 완료
86 - 완료
87 - 완료
88 - 완료
89 - 완료
90 - 완료
91 - 완료
92 - 완료
93 - 완료
94 - 완료
95 - 완료
96 - 완료
97 - 완료
98 - 완료
99 - 완료
100 - 완료
101 - 완료
102 - 완료
103 - 완료
104 - 완료
105 - 완료
106 - 완료
107 - 완료
108 - 완료
109 - 완료
110 - 완료
111 - 완료
112 - 완료
113 - 완료
114 - 완료
115 - 완료
116 - 완료
117 - 완료
118 - 완료
119 - 완료
120 - 완료
121 - 완료
122 - 완료
123

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
len(df2)

6385

In [34]:
(df2.token != '').sum()

274

In [42]:
(df2.token == '').sum()

6110

In [44]:
non_count = (df2.token != '').sum()

In [45]:
non_count

275

In [ ]:
while non_count !=0:
    try:
        for i in range(non_count,len(df2)):
            temp = []
            checked = []

            temp = okt.nouns(df2['text'].iloc[i])
            temp = [word for word in temp if not word in stopwords] # 불용어 제거
            temp = [word for word in temp if len(word) > 1] # 불용어 제거
        #     print(i,'번 토큰화, 불용어 제거 완료')

            temp = list(set(temp))
        #     print(i, ' 중복단어 제거')
            for check in temp:
                aa = spell_checker.check(check).checked
                checked.append(aa)
        #     print(i,'- spell check')

            df2['token'].iloc[i] = checked
            print(i,'- 완료')
        break
    except:
        non_count = (df2.token != '').sum()
        print(non_count,' 부터 다시시작')

275 - 완료
276 - 완료
277 - 완료
278 - 완료
279 - 완료
280 - 완료
281 - 완료
282 - 완료
283 - 완료
284 - 완료
285 - 완료
286 - 완료
287 - 완료
288 - 완료
289 - 완료
290 - 완료
291 - 완료
292 - 완료
293 - 완료
294 - 완료
295 - 완료
296 - 완료
297 - 완료
298 - 완료
299 - 완료
300 - 완료
301 - 완료
302 - 완료
303 - 완료
304 - 완료
305 - 완료
306 - 완료
307 - 완료
308 - 완료
309 - 완료
310 - 완료
311 - 완료
312 - 완료
313 - 완료
314 - 완료
315 - 완료
316 - 완료
317 - 완료
318 - 완료
319 - 완료
320 - 완료
321 - 완료
322 - 완료
323 - 완료
324 - 완료
325 - 완료
326 - 완료
327 - 완료
328 - 완료
329 - 완료
330 - 완료
331 - 완료
332 - 완료
333 - 완료
334 - 완료
335 - 완료
336 - 완료
337 - 완료
338 - 완료
339 - 완료
340 - 완료
341 - 완료
342 - 완료
343 - 완료
344 - 완료
345 - 완료
346 - 완료
347 - 완료
348 - 완료
349 - 완료
350 - 완료
351 - 완료
352 - 완료
353 - 완료
354 - 완료
355 - 완료
356 - 완료
357 - 완료
358 - 완료
359 - 완료
360 - 완료
361 - 완료
362 - 완료
363 - 완료
364 - 완료
365 - 완료
366 - 완료
367 - 완료
368 - 완료
369 - 완료
370 - 완료
371 - 완료
372 - 완료
373 - 완료
374 - 완료
375 - 완료
376 - 완료
377 - 완료
378 - 완료
379 - 완료
380 - 완료
381 - 완료
382 - 완료
383 - 완료
384 - 완료
385 - 완료
3

In [ ]:
df2.to_csv('result_df.csv', index=False)

In [ ]:
j = 0
label = [0] * len(df2)

my_dic = {"text":[], "label":label}
texts = df2.token


for df in range(len(df2)):    
    txt_data = ','.join(texts[df]).replace(',', ' ')
    
    my_dic['text'].append(txt_data) 

    for i in range(len(posneg)): 
        posflag = False 
        negflag = False 

        if i < (len(positive)-1): 
            if txt_data.find(posneg[i]) != -1: 
                posflag = True 
                print(i, "positive?","테스트 : ",txt_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i) 
                break

        if i > (len(positive)-2): 
            if txt_data.find(posneg[i]) != -1: 
                negflag = True 
                print(i, "negative?","테스트 : ",txt_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i) 
                break

    if posflag == True: 
        label[j] = 1 
        # print("positive", j) 

    elif negflag == True: 
        label[j] = -1 
        # print("negative", j) 

    elif negflag == False and posflag == False: 
        label[j] = 0 
        # print("objective", j) 

    j = j + 1
                
my_dic['label'] = label
my_df = pd.DataFrame(my_dic)

In [ ]:
my_df

In [ ]:
my_df.label.value_counts()